In [12]:
# Import necessary libraries
import cv2
import os

# Initialize the camera
cap = cv2.VideoCapture(0)

# Create a directory for storing the dataset
if not os.path.exists("dataset"):
    os.makedirs("dataset")

# Create a dataset
count = 0
while True:
    ret, frame = cap.read()
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break
    if cv2.waitKey(1) == ord('s'):
        count += 1
        filename = "dataset/user_" + str(count) + ".jpg"
        cv2.imwrite(filename, frame)
cap.release()
cv2.destroyAllWindows()


In [6]:
# Import necessary libraries
import face_recognition
import os
import numpy as np

# Create a directory for storing the embeddings
if not os.path.exists("embeddings"):
    os.makedirs("embeddings")

# Load the images
image_paths = [os.path.join("dataset", f) for f in os.listdir("dataset")]
images = []
for image_path in image_paths:
    image = face_recognition.load_image_file(image_path)
    images.append(image)

# Compute the face embeddings
embeddings = []
for image in images:
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    if len(face_encodings) == 1:
        embeddings.append(face_encodings[0])

# Save the embeddings
np.savetxt("embeddings/embeddings.txt", embeddings)


In [ ]:
# Import necessary libraries
import cv2
import face_recognition
import numpy as np
import os
import json
from datetime import datetime


In [ ]:
# Load the embeddings
embeddings = np.loadtxt("embeddings/embeddings.txt")


In [ ]:
# Initialize the camera
cap = cv2.VideoCapture(0)


In [ ]:
# Load the attendance log
if not os.path.exists("attendance.json"):
    with open("attendance.json", "w") as f:
        json.dump({}, f)
with open("attendance.json", "r") as f:
    attendance = json.load(f)


In [ ]:
# Recognize the faces and mark attendance
while True:
    ret, frame = cap.read()
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(embeddings, face_encoding)
        if True in matches:
            index = matches.index(True)
            name = "user_" + str(index+1)
            if name not in attendance:
                attendance[name] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                with open("attendance.json", "w") as f:
                    json.dump(attendance, f)
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



In [ ]:
import pandas as pd

# Load the attendance log from the JSON file
with open('attendance.json', 'r') as f:
    attendance_log = json.load(f)

# Convert the attendance log to a Pandas dataframe
df = pd.DataFrame(attendance_log)

# Write the attendance data to an Excel file
writer = pd.ExcelWriter('attendance_log.xlsx')
df.to_excel(writer, index=False)
writer.save()

